# SBA Loan Default Prediction - Data Preprocessing Pipeline

**Purpose:** Clean and prepare SBA 7(a) loan data for machine learning models  
**Dataset:** foia-7a-fy2020-present-asof-250930.csv  
**Date:** December 2025

---

## Pipeline Overview

1. **Environment Setup** - Import libraries and configure settings
2. **Data Loading** - Load and inspect the dataset
3. **Initial Exploration** - Understand data quality and structure
4. **Target Definition** - Define what we're predicting (PIF vs CHGOFF)
5. **Handle Missing Values** - Drop nulls and clean categorical variables
6. **Feature Engineering** - Create new predictive features
7. **Categorical Encoding** - One-hot encode categorical variables
8. **Drop Irrelevant Features** - Remove unnecessary columns
9. **Final Preparation** - Create final X and y for modeling
10. **Save Processed Data** - Export for modeling

**Naming Convention:** All engineered features use **PascalCase**

## 1. Environment Setup

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
%matplotlib inline

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Data Loading

In [5]:
# Load the SBA loan dataset
df = pd.read_csv('data/raw/foia-7a-fy2020-present-asof-250930.csv')

print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumn names:\n{df.columns.tolist()}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/foia-7a-fy2020-present-asof-250930.csv'

In [ ]:
# Display first few rows
df.head()

,AsOfDate,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,LocationID,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFY,FirstDisbursementDate,ProcessingMethod,Subprogram,InitialInterestRate,FixedorVariableInterestRate,TerminMonths,NAICSCode,NAICSDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidinFullDate,ChargeoffDate,GrossChargeoffAmount,RevolverStatus,JobsSupported,CollateralInd,SoldSecondMarketInd
0,2025-09-30,7A,MID-KAM CRUSHING LLC,9000 Aetna Rd,Cleveland,OH,44105,57328.0,The Huntington National Bank,6560.0,NaN,17 S High St.,COLUMBUS,OH,43215.0,82900.0,41450.0,10/01/2019,2020,NaN,SBA Express Program,FA$TRK (Small Loan Express),6.05,F,60,238910,Site Preparation Contractors,NaN,NaN,CUYAHOGA,OH,CLEVELAND DISTRICT OFFICE,11.0,CORPORATION,Existing or more than 2 years old,CANCLD,NaN,NaN,0.0,0,4.0,N,NaN
1,2025-09-30,7A,Yeadam,1428 CONTRA COSTA BLVD,PLEASANT HILL,CA,94523,433106.0,Poppy Bank,57903.0,NaN,"111 Santa Rosa Ave, Ste 320",Santa Rosa,CA,95404.0,1504500.0,1128375.0,10/01/2019,2020,10/31/2019,Preferred Lenders Program,Guaranty,6.75,V,300,722511,Full-Service Restaurants,NaN,NaN,CONTRA COSTA,CA,SAN FRANCISCO DISTRICT OFFICE,11.0,CORPORATION,Existing or more than 2 years old,PIF,06/30/2023,NaN,0.0,0,12.0,Y,NaN
2,2025-09-30,7A,Dreadnought Brewing LLC,16726 146TH ST SE STE 153,MONROE,WA,98272,59698.0,Columbia Bank,17266.0,NaN,"5885 Meadows, Ste 400",Lake Oswego,OR,97035.0,25000.0,12500.0,10/01/2019,2020,12/15/2020,SBA Express Program,FA$TRK (Small Loan Express),7.50,V,120,312120,Breweries,NaN,NaN,SNOHOMISH,WA,SEATTLE DISTRICT OFFICE,1.0,CORPORATION,Existing or more than 2 years old,EXEMPT,NaN,NaN,0.0,1,0.0,N,NaN
3,2025-09-30,7A,Big Oaks Golf Course LLC,3451 Big Oaks Golf Course,Saltillo,MS,38866,39848.0,Cadence Bank,11813.0,NaN,201 S Spring St,TUPELO,MS,38804.0,640000.0,480000.0,10/01/2019,2020,NaN,Preferred Lenders Program,Guaranty,5.75,V,240,713910,Golf Courses and Country Clubs,NaN,NaN,LEE,MS,MISSISSIPPI DISTRICT OFFICE,1.0,CORPORATION,"Startup, Loan Funds will Open Business",CANCLD,NaN,NaN,0.0,0,12.0,Y,NaN
4,2025-09-30,7A,Stillpoint Therapies LLC,"2469 Earl Campbell Parkway, Su",Tyler,TX,75701,70468.0,"Austin Bank, Texas National Association",3276.0,NaN,200 E Commerce St,JACKSONVILLE,TX,75766.0,295500.0,221625.0,10/01/2019,2020,10/31/2019,7a General,Guaranty,8.00,V,120,621399,Offices of All Other Miscellaneous Health Prac...,NaN,NaN,SMITH,TX,DALLAS / FT WORTH DISTRICT OFFICE,1.0,CORPORATION,"Startup, Loan Funds will Open Business",EXEMPT,NaN,NaN,0.0,0,5.0,Y,NaN


## 3. Initial Data Exploration

In [ ]:
# Check data types and missing values
print("="*80)
print("DATA QUALITY SUMMARY")
print("="*80)

missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Data_Type': df.dtypes.values,
    'Non_Null': df.count().values,
    'Null_Count': df.isnull().sum().values,
    'Null_Pct': (df.isnull().sum() / len(df) * 100).round(2).values
})

# Show only columns with missing values
print("\nColumns with Missing Values:")
print(missing_summary[missing_summary['Null_Count'] > 0].to_string(index=False))

DATA QUALITY SUMMARY

Columns with Missing Values:
                     Column Data_Type  Non_Null  Null_Count  Null_Pct
                   BorrName    object    347512           2      0.00
                 LocationID   float64    347021         493      0.14
             BankFDICNumber   float64    310724       36790     10.59
             BankNCUANumber   float64     10018      337496     97.12
                 BankStreet    object    347021         493      0.14
                   BankCity    object    347021         493      0.14
                  BankState    object    347021         493      0.14
                    BankZip   float64    347021         493      0.14
      FirstDisbursementDate    object    282035       65479     18.84
        InitialInterestRate   float64    347508           6      0.00
FixedorVariableInterestRate    object    347508           6      0.00
           NAICSDescription    object    321692       25822      7.43
              FranchiseCode    object  

## 4. Target Variable Definition

**Goal:** Predict loan default (CHGOFF) vs successful repayment (PIF)

**Strategy:** Filter to keep only completed loans with clear outcomes

In [ ]:
# Check LoanStatus distribution
print("="*80)
print("TARGET VARIABLE: LoanStatus Distribution")
print("="*80)
print(df['LoanStatus'].value_counts())
print(f"\nTotal unique statuses: {df['LoanStatus'].nunique()}")

TARGET VARIABLE: LoanStatus Distribution
LoanStatus
EXEMPT    226083
PIF        51791
CANCLD     40344
COMMIT     25132
CHGOFF      4163
Name: count, dtype: int64

Total unique statuses: 5


In [ ]:
# Filter to keep only completed loans (PIF = Paid in Full, CHGOFF = Charged Off)
# Rationale: We want to predict loan outcomes, not intermediate states
print("→ Filtering to keep only PIF and CHGOFF loans...")
df_filtered = df[df['LoanStatus'].isin(['PIF', 'CHGOFF'])].copy()

print(f"Rows before filtering: {len(df):,}")
print(f"Rows after filtering: {len(df_filtered):,}")
print(f"Rows removed: {len(df) - len(df_filtered):,} ({((len(df) - len(df_filtered))/len(df)*100):.2f}%)")

# Update working dataframe
df = df_filtered

→ Filtering to keep only PIF and CHGOFF loans...
Rows before filtering: 347,514
Rows after filtering: 55,954
Rows removed: 291,560 (83.90%)


In [ ]:
# Check missing values AFTER filtering to PIF/CHGOFF
# This helps us understand which columns need handling
missing_df = pd.DataFrame({
    'column': df.columns,
    'null_count': df.isnull().sum().values,
    'null_pct': (df.isnull().sum() / len(df) * 100).round(2).values
})
print("Columns with Missing Values (after filtering):")
print(missing_df[missing_df['null_count'] > 0].to_string(index=False))

Columns with Missing Values (after filtering):
               column  null_count  null_pct
           LocationID         120      0.21
       BankFDICNumber        5104      9.12
       BankNCUANumber       54125     96.73
           BankStreet         120      0.21
             BankCity         120      0.21
            BankState         120      0.21
              BankZip         120      0.21
FirstDisbursementDate           3      0.01
     NAICSDescription        1609      2.88
        FranchiseCode       48714     87.06
        FranchiseName       48740     87.11
          BusinessAge          88      0.16
       PaidinFullDate        4163      7.44
        ChargeoffDate       51795     92.57
  SoldSecondMarketInd       38096     68.08


## 5. Handling Missing Values

**Critical:** Handle missing values BEFORE feature engineering to avoid errors

**Strategy:**
- Drop rows with missing FirstDisbursementDate (only 3 rows)
- Drop rows with missing critical identifiers (< 1% data loss)
- Clean categorical variables (BusinessType, BusinessAge)

In [ ]:
# 5.1: Drop rows with missing FirstDisbursementDate
# After filtering to PIF/CHGOFF, only 3 rows have missing dates - drop them
# NOTE: We do NOT impute since we're dropping only 3 rows
print("[5.1] Dropping rows with missing FirstDisbursementDate...")
print(f"Missing FirstDisbursementDate: {df['FirstDisbursementDate'].isna().sum()}")
rows_before = len(df)
df = df.dropna(subset=['FirstDisbursementDate'])
print(f"Rows dropped: {rows_before - len(df):,}")

[5.1] Dropping rows with missing FirstDisbursementDate...
Missing FirstDisbursementDate: 3
Rows dropped: 3


In [ ]:
# 5.2: Drop rows with missing critical identifiers (0.21% - noise)
# Rationale: LocationID and BankState are needed for feature engineering
print("[5.2] Dropping rows with missing LocationID or BankState...")
rows_before = len(df)
df = df.dropna(subset=['LocationID', 'BankState'])
print(f"Rows dropped: {rows_before - len(df):,}")

[5.2] Dropping rows with missing LocationID or BankState...
Rows dropped: 120


In [ ]:
# 5.3: Clean BusinessType (whitespace and NaN)
# Strategy: Replace with 'INDIVIDUAL' (most common for small businesses)
print("[5.3] Cleaning BusinessType column...")
print(f"Unique values before: {df['BusinessType'].nunique()}")
df['BusinessType'] = df['BusinessType'].replace('        ', 'INDIVIDUAL')
df['BusinessType'] = df['BusinessType'].fillna('INDIVIDUAL')
print(f"Unique values after: {df['BusinessType'].nunique()}")
print(df['BusinessType'].value_counts())

[5.3] Cleaning BusinessType column...
Unique values before: 4
Unique values after: 3
BusinessType
CORPORATION    51746
INDIVIDUAL      3452
PARTNERSHIP      633
Name: count, dtype: int64


In [ ]:
# 5.4: Clean BusinessAge -> BusinessAge_Clean (PascalCase)
# Strategy: Map to standardized categories and handle NaN
print("[5.4] Cleaning BusinessAge column...")
print("Original distribution:")
print(df['BusinessAge'].value_counts())

age_mapping = {
    'Existing or more than 2 years old': 'Existing',
    'Startup, Loan Funds will Open Business': 'Startup',
    'New Business or 2 years or less': 'NewBusiness',
    'Change of Ownership': 'ChangeOfOwnership',
    'Unanswered': 'Existing',  # Conservative assumption
}
df['BusinessAge_Clean'] = df['BusinessAge'].map(age_mapping)
df['BusinessAge_Clean'] = df['BusinessAge_Clean'].fillna('Existing')

print("\nCleaned distribution:")
print(df['BusinessAge_Clean'].value_counts())

[5.4] Cleaning BusinessAge column...
Original distribution:
BusinessAge
Existing or more than 2 years old         30264
New Business or 2 years or less            9016
Startup, Loan Funds will Open Business     8521
Change of Ownership                        5674
Unanswered                                 2268
Name: count, dtype: int64

Cleaned distribution:
BusinessAge_Clean
Existing             32620
NewBusiness           9016
Startup               8521
ChangeOfOwnership     5674
Name: count, dtype: int64


In [ ]:
print(f"\n✓ Missing value handling complete. Final shape: {df.shape}")


✓ Missing value handling complete. Final shape: (55831, 44)


## 6. Feature Engineering

Create new features AFTER handling missing values to avoid errors

**Naming Convention:** All features use **PascalCase**

In [ ]:
# 6.1: Convert date columns to datetime (required for calculations)
print("[6.1] Converting date columns to datetime format...")
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], errors='coerce')
df['FirstDisbursementDate'] = pd.to_datetime(df['FirstDisbursementDate'], errors='coerce')
print("✓ Date conversion complete")

[6.1] Converting date columns to datetime format...
✓ Date conversion complete


In [ ]:
# 6.2: Time-based features (PascalCase)
print("[6.2] Creating time-based features...")

# Time from approval to disbursement (in days)
df['TimeToDisbursementDays'] = (df['FirstDisbursementDate'] - df['ApprovalDate']).dt.days
df['TimeToDisbursementDays'] = df['TimeToDisbursementDays'].clip(lower=0)  # No negative values

# Days since approval (loan age/vintage)
snapshot_date = df['ApprovalDate'].max()
df['DaysSinceApproval'] = (snapshot_date - df['ApprovalDate']).dt.days

print(f"TimeToDisbursementDays - Mean: {df['TimeToDisbursementDays'].mean():.2f}, Max: {df['TimeToDisbursementDays'].max()}")
print(f"DaysSinceApproval - Mean: {df['DaysSinceApproval'].mean():.2f}, Max: {df['DaysSinceApproval'].max()}")

[6.2] Creating time-based features...
TimeToDisbursementDays - Mean: 31.93, Max: 1960
DaysSinceApproval - Mean: 1470.76, Max: 2101


In [ ]:
# 6.3: COVID-19 period indicator (PascalCase)
print("[6.3] Creating COVID-19 period indicator...")
df['IsCovidEra'] = (
    (df['ApprovalDate'] >= '2020-03-01') & 
    (df['ApprovalDate'] <= '2021-12-31')
).astype(int)
print(f"COVID loans: {df['IsCovidEra'].sum():,} ({df['IsCovidEra'].sum()/len(df)*100:.2f}%)")

[6.3] Creating COVID-19 period indicator...
COVID loans: 29,537 (52.90%)


In [ ]:
# 6.4: Loan term in years (PascalCase) - more interpretable than months
print("[6.4] Converting TerminMonths to TermInYears...")
df['TermInYears'] = (df['TerminMonths'] / 12).round(2)
print(f"Term range: {df['TermInYears'].min():.2f} to {df['TermInYears'].max():.2f} years")

[6.4] Converting TerminMonths to TermInYears...
Term range: 0.00 to 27.92 years


In [ ]:
# 6.5: NAICS Sector (first 2 digits = industry sector) (PascalCase)
print("[6.5] Extracting NAICS Sector...")
df['NAICSSector'] = df['NAICSCode'].astype(str).str[:2]
print(f"Unique sectors: {df['NAICSSector'].nunique()}")

[6.5] Extracting NAICS Sector...
Unique sectors: 24


In [ ]:
# 6.6: Binary indicator features (ALL PascalCase)
print("[6.6] Creating binary indicator features...")

# Is it a credit union?
df['IsCreditUnion'] = df['BankNCUANumber'].notna().astype(int)

# Is it a franchise?
df['IsFranchise'] = df['FranchiseCode'].notna().astype(int)

# Fixed vs variable interest rate
df['IsFixedRate'] = (df['FixedorVariableInterestRate'] == 'F').astype(int)

# Has collateral?
df['HasCollateral'] = (df['CollateralInd'] == 'Y').astype(int)

# Sold to secondary market?
df['SoldSecondaryMarket'] = np.where(df['SoldSecondMarketInd'] == 'Y', 1, 0)

print(f"IsCreditUnion: {df['IsCreditUnion'].sum():,} ({df['IsCreditUnion'].mean()*100:.2f}%)")
print(f"IsFranchise: {df['IsFranchise'].sum():,} ({df['IsFranchise'].mean()*100:.2f}%)")
print(f"IsFixedRate: {df['IsFixedRate'].sum():,} ({df['IsFixedRate'].mean()*100:.2f}%)")
print(f"HasCollateral: {df['HasCollateral'].sum():,} ({df['HasCollateral'].mean()*100:.2f}%)")
print(f"SoldSecondaryMarket: {df['SoldSecondaryMarket'].sum():,} ({df['SoldSecondaryMarket'].mean()*100:.2f}%)")

[6.6] Creating binary indicator features...
IsCreditUnion: 1,829 (3.28%)
IsFranchise: 7,218 (12.93%)
IsFixedRate: 11,465 (20.54%)
HasCollateral: 44,479 (79.67%)
SoldSecondaryMarket: 17,804 (31.89%)


In [ ]:
# 6.7: Interaction features (PascalCase)
# MUST be done BEFORE dropping raw columns
print("[6.7] Creating interaction features...")

# Same state lending (bank and project in same state)
df['SameStateLending'] = (df['BankState'] == df['ProjectState']).astype(int)
print(f"Same state loans: {df['SameStateLending'].sum():,} ({df['SameStateLending'].mean()*100:.2f}%)")

[6.7] Creating interaction features...
Same state loans: 21,698 (38.86%)


In [ ]:
# 6.8: Frequency encoding for LocationID (PascalCase)
print("[6.8] Creating frequency encoding for LocationID...")
frequency_map = df['LocationID'].value_counts()
df['LocationIDCount'] = df['LocationID'].map(frequency_map)
print(f"LocationIDCount range: {df['LocationIDCount'].min()} to {df['LocationIDCount'].max()}")

[6.8] Creating frequency encoding for LocationID...
LocationIDCount range: 1 to 4948


In [ ]:
print("\n✓ Feature engineering complete")


✓ Feature engineering complete


## 7. Categorical Encoding

One-hot encode categorical variables (automatically drops original columns)

**Prefixes:** PascalCase (`Type_`, `Age_`, `State_`)

In [ ]:
# 7.1: One-hot encode BusinessType and BusinessAge_Clean
print("[7.1] One-hot encoding BusinessType and BusinessAge_Clean...")
print(f"Columns before: {len(df.columns)}")

df = pd.get_dummies(
    df, 
    columns=['BusinessType', 'BusinessAge_Clean'], 
    prefix=['Type', 'Age'],
    dtype=int,
    drop_first=False  # Keep all categories for interpretability
)

print(f"Columns after: {len(df.columns)}")
print(f"New columns created: {[col for col in df.columns if col.startswith(('Type_', 'Age_'))]}")

[7.1] One-hot encoding BusinessType and BusinessAge_Clean...
Columns before: 56
Columns after: 61
New columns created: ['Type_CORPORATION', 'Type_INDIVIDUAL', 'Type_PARTNERSHIP', 'Age_ChangeOfOwnership', 'Age_Existing', 'Age_NewBusiness', 'Age_Startup']


In [ ]:
# 7.2: One-hot encode ProjectState
print("[7.2] One-hot encoding ProjectState...")
print(f"Unique states: {df['ProjectState'].nunique()}")
print(f"Columns before: {len(df.columns)}")

df = pd.get_dummies(
    df,
    columns=['ProjectState'],
    prefix='State',
    dtype=int,
    drop_first=False
)

print(f"Columns after: {len(df.columns)}")

[7.2] One-hot encoding ProjectState...
Unique states: 54
Columns before: 61
Columns after: 114


In [ ]:
print("\n✓ Categorical encoding complete")


✓ Categorical encoding complete


## 8. Drop Irrelevant Features

Remove columns that:
- Are raw identifiers (PII, not predictive)
- Cause data leakage (known after loan outcome)
- Have already been encoded/engineered

In [ ]:
# 8.1: Drop all irrelevant columns in one step
print("[8.1] Dropping irrelevant columns...")

cols_to_drop = [
    # Administrative/Identifier columns (PII, not predictive)
    'AsOfDate', 'Program', 'BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip',
    'BankName', 'BankStreet', 'BankCity', 'BankState', 'BankZip',
    'FranchiseName', 'ProjectCounty', 'SBADistrictOffice', 'CongressionalDistrict',
    'ProcessingMethod', 'Subprogram',
    
    # Outcome columns (data leakage - known after loan outcome)
    'PaidinFullDate', 'ChargeoffDate', 'GrossChargeoffAmount',
    
    # Raw columns already encoded/engineered
    'FranchiseCode', 'BankNCUANumber', 'BankFDICNumber',
    'NAICSCode', 'NAICSDescription',
    'FirstDisbursementDate', 'TerminMonths', 'ApprovalDate',
    'BusinessAge',
    'FixedorVariableInterestRate', 'CollateralInd', 'SoldSecondMarketInd',
    'LocationID'
]

# Only drop columns that exist in dataframe
existing_cols = [c for c in cols_to_drop if c in df.columns]
df = df.drop(columns=existing_cols)
print(f"Dropped {len(existing_cols)} columns")
print(f"Remaining shape: {df.shape}")

[8.1] Dropping irrelevant columns...
Dropped 34 columns
Remaining shape: (55831, 80)


## 9. Final Data Preparation

Create the final feature matrix (X) and target vector (y)

In [ ]:
# 9.1: Create target variable
print("[9.1] Creating target variable...")
# PIF (Paid in Full) = 0 (Good), CHGOFF (Charged Off) = 1 (Bad)
y = df['LoanStatus'].map({'PIF': 0, 'CHGOFF': 1})

print(f"Target distribution:")
print(f"  Good loans (PIF=0): {(y == 0).sum():,} ({(y == 0).mean()*100:.2f}%)")
print(f"  Bad loans (CHGOFF=1): {(y == 1).sum():,} ({(y == 1).mean()*100:.2f}%)")

# Create feature set (drop target from features)
X = df.drop(columns=['LoanStatus'])

[9.1] Creating target variable...
Target distribution:
  Good loans (PIF=0): 51,669 (92.55%)
  Bad loans (CHGOFF=1): 4,162 (7.45%)


In [ ]:
# 9.2: Verify data quality
print("[9.2] Verifying data quality...")
missing_cols = X.columns[X.isnull().any()].tolist()
if missing_cols:
    print(f"⚠ Columns with missing values: {missing_cols}")
else:
    print("✓ No missing values in feature set")

[9.2] Verifying data quality...
✓ No missing values in feature set


In [ ]:
# 9.3: Final summary
print("\n" + "="*60)
print("FINAL FEATURE SET")
print("="*60)
print(f"Features: {X.shape[1]}")
print(f"Samples: {X.shape[0]:,}")
print(f"\nFeature names:")
for i, col in enumerate(X.columns, 1):
    print(f"  {i:2d}. {col}")


FINAL FEATURE SET
Features: 79
Samples: 55,831

Feature names:
   1. GrossApproval
   2. SBAGuaranteedApproval
   3. ApprovalFY
   4. InitialInterestRate
   5. RevolverStatus
   6. JobsSupported
   7. TimeToDisbursementDays
   8. DaysSinceApproval
   9. IsCovidEra
  10. TermInYears
  11. NAICSSector
  12. IsCreditUnion
  13. IsFranchise
  14. IsFixedRate
  15. HasCollateral
  16. SoldSecondaryMarket
  17. SameStateLending
  18. LocationIDCount
  19. Type_CORPORATION
  20. Type_INDIVIDUAL
  21. Type_PARTNERSHIP
  22. Age_ChangeOfOwnership
  23. Age_Existing
  24. Age_NewBusiness
  25. Age_Startup
  26. State_AK
  27. State_AL
  28. State_AR
  29. State_AZ
  30. State_CA
  31. State_CO
  32. State_CT
  33. State_DC
  34. State_DE
  35. State_FL
  36. State_GA
  37. State_GU
  38. State_HI
  39. State_IA
  40. State_ID
  41. State_IL
  42. State_IN
  43. State_KS
  44. State_KY
  45. State_LA
  46. State_MA
  47. State_MD
  48. State_ME
  49. State_MI
  50. State_MN
  51. State_MO
  52. 

In [ ]:
print("\n" + "="*60)
print("✓ DATA PREPROCESSING COMPLETE")
print("="*60)
print(f"\nReady for modeling:")
print(f"  X (features): {X.shape}")
print(f"  y (target): {y.shape}")


✓ DATA PREPROCESSING COMPLETE

Ready for modeling:
  X (features): (55831, 79)
  y (target): (55831,)


## 10. Save Processed Data (Optional)

In [ ]:
# Save to CSV (uncomment to save)
X.to_csv('sba_loan_features.csv', index=False)
y.to_csv('sba_loan_target.csv', index=False, header=['IsDefault'])
print("✓ Data saved:")
print("  - sba_loan_features.csv")
print("  - sba_loan_target.csv")

✓ Data saved:
  - sba_loan_features.csv
  - sba_loan_target.csv
